In [1]:
import pandas as pd
import numpy as np

from gensim.models import Word2Vec
import math
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, LSTM, TimeDistributed, RepeatVector,Input
from keras.utils import np_utils
from keras.models import Model
from keras.wrappers.scikit_learn import KerasClassifier
from keras import backend as K
from keras.layers import Bidirectional
from keras.preprocessing.text import Tokenizer, text_to_word_sequence
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical
from keras.layers import Embedding
from keras.layers import Conv1D, MaxPooling1D, Embedding, Dropout, LSTM, GRU, Bidirectional, TimeDistributed

from keras.engine.topology import Layer, InputSpec
from keras import initializers
from sklearn.model_selection import train_test_split

#參數
MAX_SENT_LENGTH = 100
MAX_SENTS = 15
MAX_NB_WORDS = 512
EMBEDDING_DIM = 300
VALIDATION_SPLIT = 0.1



column_names = ['type','title','text']
df = pd.read_csv('./all_after_mapping.tsv',sep='\t',names=column_names)
labels = df['type'].values
labels = np.array(labels)
np.save('labels',labels)

tokenlizeword = np.load('./tokenlizeword0225_nopunct.npy',allow_pickle=True)
wmodel=Word2Vec.load('./word2vec.model')


# embedding
x_train = []
for k in range(tokenlizeword.shape[0]):
# every article have max_size * 300 embedding matrix
    embedding_matrix = np.zeros((MAX_NB_WORDS,300))
    for i in range(len(tokenlizeword[k])):
        if(i>=MAX_NB_WORDS):
            break
        embedding_matrix[i] = wmodel[tokenlizeword[k][i]]
    x_train.append(embedding_matrix)

print("embedding done")

print(embedding_matrix)
embedding_layer = Embedding(MAX_NB_WORDS,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SENT_LENGTH,
                            trainable=True,
                            mask_zero=True)
x_train = np.array(x_train)
# print(x_train.shape)
x = x_train
y = labels
print(y.shape)
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.1,random_state=5)
y_train = np_utils.to_categorical(y_train)

# !ls
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)



sentence_input = Input(shape=(MAX_SENT_LENGTH,), dtype='int32')
embedded_sequences = embedding_layer(sentence_input)
l_lstm = Bidirectional(LSTM(100))(embedded_sequences)
sentEncoder = Model(sentence_input, l_lstm)

review_input = Input(shape=(MAX_SENTS,MAX_SENT_LENGTH), dtype='int32')
review_encoder = TimeDistributed(sentEncoder)(review_input)
l_lstm_sent = Bidirectional(LSTM(100))(review_encoder)
preds = Dense(2, activation='softmax')(l_lstm_sent)
model = Model(review_input, preds)

print("model fitting - Hierachical attention network")
model.fit(x_train, y_train, validation_split=0.2,
          epoch=3, batch_size=50)
pred = model.predict(x_test)
pred = np.argmax(pred, axis = 1) # Decode softmax output
accuracy = (pred == y_test).mean()
print('Your test accuracy is %.6f' % (accuracy * 100))

Using TensorFlow backend.


FileNotFoundError: [Errno 2] File b'./all_after_mapping.tsv' does not exist: b'./all_after_mapping.tsv'